In [1]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from sumolib import checkBinary
import torch.nn.functional as F
from utils import *
from Models import GRUModel
from Datasets import DatasetWithPlans
from CustomMSE import CustomMSE

# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

SUMO_HOME found
Using device: cuda


In [2]:
planned_path = get_planned_path()
checkpoints = list(planned_path.values())
checkpoints = torch.tensor(checkpoints).float() / 10

 Retrying in 1 seconds
***Starting server on port 40095 ***
Loading net-file from './config/osm.net.xml.gz' ... done (103ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.
Simulation ended at time: 10777.00
Reason: TraCI requested termination.
Performance: 
 Duration: 8.49s
 TraCI-Duration: 5.10s
 Real time factor: 1268.93
 UPS: 137908.630637
Vehicles: 
 Inserted: 2199
 Running: 0
 Waiting: 0
Statistics (avg of 2199):
 RouteLength: 4984.74
 Speed: 9.48
 Duration: 532.63
 WaitingTime: 19.92
 TimeLoss: 76.98
 DepartDelay: 0.53



In [3]:
df = pd.read_csv('edinburgh_trajectories.csv')
num_cols = df.shape[1]
position_indices = [i for i in range(num_cols) if i % 4 == 1 or i % 4 == 2]
position_df = df.iloc[:, position_indices]
position_array = position_df.to_numpy()
sequence_length = len(position_indices) // 2
tensor_list = []

for row in position_array:
    reshaped_tensor = torch.tensor(row.reshape(sequence_length, 2))
    tensor_list.append(reshaped_tensor)

all_trajectories_tensor = torch.stack(tensor_list).float() / 10

next_checkpoint = torch.zeros_like(all_trajectories_tensor)
next_next_checkpoint = torch.zeros_like(all_trajectories_tensor)
checkpoints_pad_1 = F.pad(checkpoints, (0, 0, 1, 0))
for i in range(all_trajectories_tensor.shape[1]):
    _, min_indices = project_to_nearest(all_trajectories_tensor[:, i], checkpoints)
    next_checkpoint[:, i] = checkpoints_pad_1[range(checkpoints.shape[0]), min_indices+1]
    next_next_checkpoint[:, i] = checkpoints_pad_1[range(checkpoints.shape[0]), min_indices+2]

all_trajectories_tensor = torch.cat((all_trajectories_tensor, next_checkpoint, next_next_checkpoint), dim=2)

In [4]:
# break trajectories into 1000 by 1000 cells
trajectory_list = []
checkpoints_list = []
for i in range(all_trajectories_tensor.shape[0]):
    trajectory = all_trajectories_tensor[i]
    path = checkpoints[i]
    modif = (trajectory[0, :2] - torch.tensor([50, 50])).repeat(1, 3)
    to_add = []
    for j in range(trajectory.shape[0]):
        trajectory_j = (trajectory[j] - modif).squeeze()
        # break if end of trajectory =============================
        if trajectory[j, :2].max() == 0: 
            if trajectory[j:, :2].max() == 0:
                break
        # ========================================================
        # if vehicle moves out of the previous box, add the trajectory and checkpoints.
        if trajectory_j[0] < 0 or trajectory_j[0] > 100 or trajectory_j[1] < 0 or trajectory_j[1] > 100 or (j == trajectory.shape[0] - 1):
            # add the trajectory to list
            trajectory_list.append(torch.stack(to_add))
            checkpoints_list.append(path-modif.squeeze()[:2])

            # reset the to_add list, adding a part of the history to the new trajectory
            if j != trajectory.shape[0] - 1:
                modif = (trajectory[j, :2] - torch.tensor([50, 50])).repeat(1, 3)
                rewind = 0
                while True:
                    trajectory_k = (trajectory[j - rewind] - modif).squeeze()
                    if trajectory_k[0] < 0 or trajectory_k[0] > 100 or trajectory_k[1] < 0 or trajectory_k[1] > 100:
                        rewind += 1
                    else:
                        break
                to_add = to_add[:-rewind]
        else:
            to_add.append(trajectory_j)

In [5]:
checkpoints = torch.stack(checkpoints_list)
def pad_tensors(tensors):
    max_length = max([t.shape[0] for t in tensors])
    padded_tensors = []
    for tensor in tensors:
        padded_tensor = F.pad(tensor, (0, 0, 0, max_length - tensor.shape[0]))
        padded_tensors.append(padded_tensor)
    return torch.stack(padded_tensors)

all_trajectories_tensor = pad_tensors(trajectory_list)

In [6]:
# split the data into training and validation sets
# because the data is randomly generated, we don't need to shuffle it
train_ratio = 0.8
train_size = int(train_ratio * all_trajectories_tensor.shape[0])
train_trajectories_tensor = all_trajectories_tensor[:train_size]
val_trajectories_tensor = all_trajectories_tensor[train_size:]

train_checkpoints = checkpoints[:train_size]
val_checkpoints = checkpoints[train_size:]

train_mask = generate_masks(train_trajectories_tensor)
    
train_dataset = DatasetWithPlans(train_trajectories_tensor, train_mask, train_checkpoints)

In [7]:
def train_model(model, dataloader, epochs, optimizer, criterion):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, masks, planned_path in dataloader:
            optimizer.zero_grad()
            hidden = None
            loss = 0
            steps = 0
            # Autoregressive prediction
            # Start with the first input and predict each subsequent step
            seq_len = inputs.size(1)
            current_input = inputs[:, 0, :].unsqueeze(1)
            modifications = (current_input[:, :, :2] - torch.ones_like(current_input[:, :, :2]) * 500).repeat(1, 1, 3)
            current_input -= modifications
            for t in range(1, seq_len):
                new_steps = int(sum((inputs[:, t, :2].reshape(-1) != 0).float().to(device))) / 2
                if new_steps == 0:
                    break
                prediction, hidden = model(current_input, hidden)
                steps += new_steps

                previous_input = inputs[:, t-1, :2]
                if epoch > 10:
                    projection_with_checkpoints = project_to_nearest_with_checkpoints(prediction, planned_path)
                    current_input = (projection_with_checkpoints * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
                else:
                    current_input = inputs[:, t, :].unsqueeze(1)
                if t > 1:
                    loss += criterion(prediction - previous_input, inputs[:, t, :2]-previous_input) #(current_input-previous_input).squeeze(1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
 
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

model = GRUModel(input_size=6, hidden_size=256, num_layers=2, output_size=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CustomMSE()
train_model(model, train_dataloader, 20, optimizer, criterion)

Epoch 1, Loss: 64560.91575734992
Epoch 2, Loss: 3529.0958406147206
Epoch 3, Loss: 3349.6295127467106
Epoch 4, Loss: 3221.3693449321545
Epoch 5, Loss: 3001.158968955592
Epoch 6, Loss: 3116.736734169408
Epoch 7, Loss: 2992.0828716077303
Epoch 8, Loss: 2955.343814247533
Epoch 9, Loss: 2959.184515059622
Epoch 10, Loss: 2957.4387200606498
Epoch 11, Loss: 2879.2032432154606
Epoch 12, Loss: 2963.9348388671874
Epoch 13, Loss: 2937.0258210834704
Epoch 14, Loss: 2864.696548622533
Epoch 15, Loss: 2887.2699584960938
Epoch 16, Loss: 2912.6156808953538
Epoch 17, Loss: 2818.1108199270147
Epoch 18, Loss: 2903.3135478772615
Epoch 19, Loss: 2711.706683670847
Epoch 20, Loss: 2895.953903037623


In [12]:
def checkModel(model, inputs, masks, paths):
    model.eval()
    with torch.no_grad():
        hidden = None
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            prediction, hidden = model(current_input, hidden)
            print(prediction)
            projection_with_checkpoints = project_to_nearest_with_checkpoints(prediction, paths)
            current_input = (projection_with_checkpoints * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            # print(projection_with_checkpoints[0][:2]*10, inputs[:, t, :][0][:2]*10)

# get the first batch in dataloader
val_mask = generate_masks(val_trajectories_tensor, missing_ratio=0.9, complete_traj_ratio=0)
val_dataset = DatasetWithPlans(val_trajectories_tensor, val_mask, val_checkpoints)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True)
inputs, masks, paths = next(iter(val_dataloader))

checkModel(model, inputs, masks, paths)

tensor([[15.1684, 20.1092]], device='cuda:0')
tensor([[55.9472, 50.3040]], device='cuda:0')
tensor([[58.6681, 51.4483]], device='cuda:0')
tensor([[61.0475, 52.3073]], device='cuda:0')
tensor([[62.5716, 52.5784]], device='cuda:0')
tensor([[64.1102, 53.5153]], device='cuda:0')
tensor([[65.5671, 53.9814]], device='cuda:0')
tensor([[66.7938, 54.3407]], device='cuda:0')
tensor([[67.8153, 54.6600]], device='cuda:0')
tensor([[68.6380, 54.8689]], device='cuda:0')
tensor([[69.2830, 55.0186]], device='cuda:0')
tensor([[69.7842, 55.1225]], device='cuda:0')
tensor([[70.1708, 55.1937]], device='cuda:0')
tensor([[70.4671, 55.2426]], device='cuda:0')
tensor([[71.0191, 54.9942]], device='cuda:0')
tensor([[71.6400, 55.6457]], device='cuda:0')
tensor([[72.1547, 55.5659]], device='cuda:0')
tensor([[72.5627, 55.6206]], device='cuda:0')
tensor([[72.9404, 55.7100]], device='cuda:0')
tensor([[73.2727, 55.7531]], device='cuda:0')
tensor([[73.5651, 55.8010]], device='cuda:0')
tensor([[73.8280, 55.8470]], devic

In [17]:
def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0
    steps = 0
    for inputs, masks, paths in dataloader:
        hidden = None
        loss = 0
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            new_steps = int(sum((inputs[:, t, :2].reshape(-1) != 0).float().to(device))) / 2
            if new_steps == 0:
                break
            steps += new_steps
            prediction, hidden = model(current_input, hidden)
            projection_with_checkpoints = project_to_nearest_with_checkpoints(prediction, paths)
            current_input = (projection_with_checkpoints * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            loss += torch.norm((projection_with_checkpoints[:, :2] - inputs[:, t, :2]) * 10, dim=1) * (inputs[:, t, :2] != 0)[:, 0]
        total_loss += loss.sum().item()
    print(f'Validation Loss: {total_loss / steps}')

for i in [0.9, 0.8, 0.7, 0.6, 0.5, 0]:
    val_mask = generate_masks(val_trajectories_tensor, missing_ratio=i, complete_traj_ratio=0)
    val_dataset = DatasetWithPlans(val_trajectories_tensor, val_mask, val_checkpoints)
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
    evaluate_model(model, val_dataloader)

Validation Loss: 63.79428057733073
Validation Loss: 46.96636624033158
Validation Loss: 40.05981302788013
Validation Loss: 36.19261911833051
Validation Loss: 33.99255013972077
Validation Loss: 30.81814166334704


In [24]:
# # save the model
# torch.save(model.state_dict(), 'gru_trajectory_prediction.pth')